# Inverse Kinematics in Robotics

This notebook demonstrates inverse kinematics (IK) concepts for robotic manipulators, complementary to the forward kinematics concepts covered in Chapter 4 of the book.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import matplotlib.animation as animation

%matplotlib inline
plt.style.use('seaborn-v0_8')

## 1. Introduction to Inverse Kinematics

While forward kinematics calculates end-effector position from joint angles, inverse kinematics (IK) solves the reverse: finding the joint angles needed to achieve a desired end-effector position. This is often more challenging and has various practical applications in robotics.

## 2. Analytical Inverse Kinematics for a 2-DOF Planar Manipulator

Let's start with a simple 2-DOF planar manipulator, for which we can derive analytical solutions.

In [ ]:
def inverse_kinematics_2dof_analytical(x, y, l1, l2):
    """
    Analytical solution for 2-DOF planar manipulator inverse kinematics.
    
    Args:
        x, y: Desired end-effector position
        l1, l2: Link lengths
    
    Returns:
        List of solutions [(theta1_1, theta2_1), (theta1_2, theta2_2)]
        Returns None if no solution exists
    """
    # Calculate distance from origin to target
    r = np.sqrt(x**2 + y**2)
    
    # Check if target is reachable
    max_reach = l1 + l2
    min_reach = abs(l1 - l2)
    
    if r > max_reach:
        print(f"Target ({x:.3f}, {y:.3f}) is out of reach (max: {max_reach:.3f}).")
        return None
    elif r < min_reach:
        print(f"Target ({x:.3f}, {y:.3f}) is inside inner workspace (min: {min_reach:.3f}).")
        return None
    
    # Calculate theta2 using law of cosines
    cos_theta2 = (l1**2 + l2**2 - r**2) / (2 * l1 * l2)
    cos_theta2 = np.clip(cos_theta2, -1, 1)  # Clamp to valid range
    theta2 = np.arccos(cos_theta2)
    
    # Two possible solutions for theta2 (elbow up and elbow down)
    theta2_1 = theta2
    theta2_2 = -theta2
    
    # Calculate theta1 for each solution
    k1_1 = l1 + l2 * np.cos(theta2_1)
    k2_1 = l2 * np.sin(theta2_1)
    theta1_1 = np.arctan2(y, x) - np.arctan2(k2_1, k1_1)
    
    k1_2 = l1 + l2 * np.cos(theta2_2)
    k2_2 = l2 * np.sin(theta2_2)
    theta1_2 = np.arctan2(y, x) - np.arctan2(k2_2, k1_2)
    
    return [(theta1_1, theta2_1), (theta1_2, theta2_2)]

def forward_kinematics_2dof(theta1, theta2, l1, l2):
    """
    Helper function to calculate forward kinematics.
    """
    # Calculate position of joint 2
    x1 = l1 * np.cos(theta1)
    y1 = l1 * np.sin(theta1)
    
    # Calculate position of end-effector
    x2 = x1 + l2 * np.cos(theta1 + theta2)
    y2 = y1 + l2 * np.sin(theta1 + theta2)
    
    return x2, y2, x1, y1

def plot_2dof_config(theta1, theta2, l1, l2, target_pos=None, title="2-DOF Manipulator Configuration"):
    """
    Plot the 2-DOF manipulator at the given configuration.
    """
    # Calculate positions
    ex, ey, jx, jy = forward_kinematics_2dof(theta1, theta2, l1, l2)
    
    plt.figure(figsize=(8, 8))
    
    # Draw links
    plt.plot([0, jx], [0, jy], 'b-', linewidth=6, label=f'Link 1 (L={l1})', alpha=0.7)
    plt.plot([jx, ex], [jy, ey], 'r-', linewidth=6, label=f'Link 2 (L={l2})', alpha=0.7)
    
    # Draw joints and end effector
    plt.plot(0, 0, 'ko', markersize=12, label='Base', zorder=5)
    plt.plot(jx, jy, 'bo', markersize=10, label='Joint 1', zorder=5)
    plt.plot(ex, ey, 'ro', markersize=10, label='End Effector', zorder=5)
    
    # Mark target if provided
    if target_pos:
        plt.plot(target_pos[0], target_pos[1], 'gs', markersize=12, label='Target', zorder=5)
    
    # Add labels
    plt.annotate(f'({ex:.2f}, {ey:.2f})', xy=(ex, ey), xytext=(10, 10), 
                 textcoords='offset points', fontsize=10,
                 bbox=dict(boxstyle='round,pad=0.3', facecolor='red', alpha=0.5))
    
    # Set axis properties
    plt.axis('equal')
    plt.grid(True, alpha=0.3)
    max_reach = l1 + l2
    plt.xlim(-max_reach-0.5, max_reach+0.5)
    plt.ylim(-max_reach-0.5, max_reach+0.5)
    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.title(f'{title}\nθ1={theta1:.3f} rad ({np.degrees(theta1):.1f}°), θ2={theta2:.3f} rad ({np.degrees(theta2):.1f}°)')
    plt.xlabel('X position')
    plt.ylabel('Y position')
    plt.legend()
    
    plt.show()

# Example usage
print("Analytical Inverse Kinematics for 2-DOF Manipulator")
print("=" * 55)

l1, l2 = 1.0, 0.8
print(f"Robot parameters: L1={l1}, L2={l2}")
print()

In [ ]:
# Test with different targets
test_targets = [
    (1.2, 0.5),   # Reachable position
    (0.8, 1.0),   # Upward position
    (1.8, 0.0),   # Extended reach (on border)
    (0.0, 0.2),   # Close to base
]

for i, (target_x, target_y) in enumerate(test_targets):
    print(f"Test {i+1}: Target at ({target_x:.3f}, {target_y:.3f})")
    
    solutions = inverse_kinematics_2dof_analytical(target_x, target_y, l1, l2)
    
    if solutions:
        print(f"Found {len(solutions)} solution(s)")
        for j, (theta1, theta2) in enumerate(solutions):
            print(f"  Solution {j+1}: θ1={theta1:.3f} rad ({np.degrees(theta1):.1f}°), θ2={theta2:.3f} rad ({np.degrees(theta2):.1f}°)")
            
            # Verify by calculating forward kinematics
            fk_x, fk_y, _, _ = forward_kinematics_2dof(theta1, theta2, l1, l2)
            error = np.sqrt((fk_x - target_x)**2 + (fk_y - target_y)**2)
            print(f"    Verification: FK gives ({fk_x:.3f}, {fk_y:.3f}), Error: {error:.6f}")
            
            # Visualize the solution
            plot_2dof_config(theta1, theta2, l1, l2, (target_x, target_y),
                           f'Solution {j+1} for target ({target_x:.1f}, {target_y:.1f})')
    
    print("-" * 60)
    print()

## 3. Numerical Inverse Kinematics

For more complex robots or when analytical solutions are not feasible, numerical methods provide a general approach to solving IK problems.

In [ ]:
def jacobian_2dof_planar(theta1, theta2, l1, l2):
    """
    Calculate the Jacobian matrix for a 2-DOF planar manipulator.
    
    Args:
        theta1, theta2: Joint angles
        l1, l2: Link lengths
    
    Returns:
        2x2 Jacobian matrix
    """
    jacobian = np.zeros((2, 2))
    
    # dx/dtheta1 = -l1*sin(theta1) - l2*sin(theta1+theta2)
    # dx/dtheta2 = -l2*sin(theta1+theta2)
    # dy/dtheta1 = l1*cos(theta1) + l2*cos(theta1+theta2)
    # dy/dtheta2 = l2*cos(theta1+theta2)
    
    jacobian[0, 0] = -l1*np.sin(theta1) - l2*np.sin(theta1 + theta2)  # dx/dtheta1
    jacobian[0, 1] = -l2*np.sin(theta1 + theta2)                       # dx/dtheta2
    jacobian[1, 0] = l1*np.cos(theta1) + l2*np.cos(theta1 + theta2)    # dy/dtheta1
    jacobian[1, 1] = l2*np.cos(theta1 + theta2)                        # dy/dtheta2
    
    return jacobian

def inverse_kinematics_jacobian_method(target_pos, initial_joints, link_lengths, 
                                     max_iterations=100, tolerance=1e-5, alpha=0.1):
    """
    Solve inverse kinematics using the Jacobian transpose method.
    
    Args:
        target_pos: Desired end-effector position (x, y)
        initial_joints: Initial joint angles [theta1, theta2]
        link_lengths: Link lengths [l1, l2]
        max_iterations: Maximum number of iterations
        tolerance: Convergence tolerance
        alpha: Learning rate/scale factor
    
    Returns:
        Joint angles that achieve the target position, or None if failed
    """
    l1, l2 = link_lengths
    target_x, target_y = target_pos
    joints = np.array(initial_joints, dtype=float)
    
    for i in range(max_iterations):
        # Calculate current end-effector position using forward kinematics
        curr_x, curr_y, _, _ = forward_kinematics_2dof(joints[0], joints[1], l1, l2)
        current_pos = np.array([curr_x, curr_y])
        
        # Calculate error
        error = np.array([target_x, target_y]) - current_pos
        
        # Check for convergence
        if np.linalg.norm(error) < tolerance:
            print(f"Converged after {i+1} iterations")
            return joints
        
        # Calculate Jacobian at current configuration
        J = jacobian_2dof_planar(joints[0], joints[1], l1, l2)
        
        # Calculate joint change using Jacobian transpose method
        # delta_theta = J^T * delta_x
        joint_change = J.T @ error * alpha
        
        # Update joint angles
        joints += joint_change
        
        # Optional: limit the amount of change in each iteration
        max_change = 0.2  # Maximum change per iteration
        if np.linalg.norm(joint_change) > max_change:
            joints -= joint_change
            joints += joint_change * max_change / np.linalg.norm(joint_change)
    
    print(f"Failed to converge after {max_iterations} iterations. Final error: {np.linalg.norm(error):.6f}")
    return None

def inverse_kinematics_pseudoinverse_method(target_pos, initial_joints, link_lengths,
                                         max_iterations=100, tolerance=1e-5, alpha=0.1):
    """
    Solve inverse kinematics using the Jacobian pseudoinverse (Moore-Penrose inverse) method.
    This method is more robust than the transpose method, especially near singularities.
    
    Args:
        target_pos: Desired end-effector position (x, y)
        initial_joints: Initial joint angles [theta1, theta2]
        link_lengths: Link lengths [l1, l2]
        max_iterations: Maximum number of iterations
        tolerance: Convergence tolerance
        alpha: Learning rate/scale factor
    
    Returns:
        Joint angles that achieve the target position, or None if failed
    """
    l1, l2 = link_lengths
    target_x, target_y = target_pos
    joints = np.array(initial_joints, dtype=float)
    
    for i in range(max_iterations):
        # Calculate current end-effector position
        curr_x, curr_y, _, _ = forward_kinematics_2dof(joints[0], joints[1], l1, l2)
        current_pos = np.array([curr_x, curr_y])
        
        # Calculate error
        error = np.array([target_x, target_y]) - current_pos
        
        # Check for convergence
        if np.linalg.norm(error) < tolerance:
            print(f"Pseudoinverse method converged after {i+1} iterations")
            return joints
        
        # Calculate Jacobian
        J = jacobian_2dof_planar(joints[0], joints[1], l1, l2)
        
        # Calculate pseudoinverse of Jacobian
        J_pinv = np.linalg.pinv(J)
        
        # Calculate joint change
        joint_change = J_pinv @ error * alpha
        
        # Update joint angles
        joints += joint_change
        
        # Limit excessive changes
        max_change = 0.2
        if np.linalg.norm(joint_change) > max_change:
            joints -= joint_change
            joints += joint_change * max_change / np.linalg.norm(joint_change)
    
    print(f"Pseudoinverse method failed after {max_iterations} iterations. Final error: {np.linalg.norm(error):.6f}")
    return None

In [ ]:
# Test numerical methods
print("Numerical Inverse Kinematics Methods")
print("=" * 40)

l1, l2 = 1.0, 0.8
target = (1.2, 0.5)
initial_guess = [0.1, 0.1]  # Small non-zero initial guess

print(f"Target position: ({target[0]:.3f}, {target[1]:.3f})")
print(f"Initial guess: θ1={initial_guess[0]:.3f}, θ2={initial_guess[1]:.3f}")
print()

# Test Jacobian transpose method
print("1. Jacobian Transpose Method:")
solution_jt = inverse_kinematics_jacobian_method(target, initial_guess, [l1, l2])

if solution_jt is not None:
    print(f"  Solution found: θ1={solution_jt[0]:.3f}, θ2={solution_jt[1]:.3f}")
    # Verify solution
    fk_x, fk_y, _, _ = forward_kinematics_2dof(solution_jt[0], solution_jt[1], l1, l2)
    final_error = np.sqrt((fk_x - target[0])**2 + (fk_y - target[1])**2)
    print(f"  Verification: FK gives ({fk_x:.3f}, {fk_y:.3f}), Error: {final_error:.6f}")
    plot_2dof_config(solution_jt[0], solution_jt[1], l1, l2, target, 
                    f'Solution via Jacobian Transpose Method')
else:
    print("  Solution not found")
print()

# Test Jacobian pseudoinverse method
print("2. Jacobian Pseudoinverse Method:")
solution_jpi = inverse_kinematics_pseudoinverse_method(target, initial_guess, [l1, l2])

if solution_jpi is not None:
    print(f"  Solution found: θ1={solution_jpi[0]:.3f}, θ2={solution_jpi[1]:.3f}")
    # Verify solution
    fk_x, fk_y, _, _ = forward_kinematics_2dof(solution_jpi[0], solution_jpi[1], l1, l2)
    final_error = np.sqrt((fk_x - target[0])**2 + (fk_y - target[1])**2)
    print(f"  Verification: FK gives ({fk_x:.3f}, {fk_y:.3f}), Error: {final_error:.6f}")
    plot_2dof_config(solution_jpi[0], solution_jpi[1], l1, l2, target, 
                    f'Solution via Jacobian Pseudoinverse Method')
else:
    print("  Solution not found")
print()

# Compare to analytical solution
print("3. Analytical Solution Comparison:")
analytical_solutions = inverse_kinematics_2dof_analytical(target[0], target[1], l1, l2)

if analytical_solutions:
    for i, (a_theta1, a_theta2) in enumerate(analytical_solutions):
        print(f"  Analytical Solution {i+1}: θ1={a_theta1:.3f}, θ2={a_theta2:.3f}")
        
        # Verify
        fk_x, fk_y, _, _ = forward_kinematics_2dof(a_theta1, a_theta2, l1, l2)
        error = np.sqrt((fk_x - target[0])**2 + (fk_y - target[1])**2)
        print(f"    Verification: FK gives ({fk_x:.3f}, {fk_y:.3f}), Error: {error:.6f}")
else:
    print("  No analytical solution found (target may be invalid)")

## 4. Singularity Analysis in Inverse Kinematics

Singularities occur when the Jacobian matrix loses rank, meaning the robot loses one or more degrees of freedom. This is problematic for numerical IK methods.

In [ ]:
def detect_singularity(jacobian, threshold=1e-3):
    """
    Detect if the Jacobian is near a singularity by calculating its determinant.
    
    Args:
        jacobian: Jacobian matrix
        threshold: Threshold below which determinant indicates singularity
    
    Returns:
        True if near singularity, False otherwise
    """
    det = np.linalg.det(jacobian)
    return abs(det) < threshold

def analyze_singularity_region(l1, l2, center=(0, 0), radius=0.1, res=20):
    """
    Analyze a region around a point to find nearby singularities.
    """
    # Create a grid around the center point
    x_vals = np.linspace(center[0]-radius, center[0]+radius, res)
    y_vals = np.linspace(center[1]-radius, center[1]+radius, res)
    X, Y = np.meshgrid(x_vals, y_vals)
    
    # Calculate if each point is near a singularity
    singularity_map = np.zeros_like(X, dtype=bool)
    det_values = np.zeros_like(X)
    
    for i in range(res):
        for j in range(res):
            x, y = X[i,j], Y[i,j]
            
            # Find an IK solution for this point
            ik_solution = inverse_kinematics_pseudoinverse_method(
                (x, y), [0.1, 0.1], [l1, l2], max_iterations=50
            )
            
            if ik_solution is not None:
                # Calculate Jacobian at this configuration
                J = jacobian_2dof_planar(ik_solution[0], ik_solution[1], l1, l2)
                det_values[i, j] = abs(np.linalg.det(J))
                singularity_map[i, j] = detect_singularity(J)
            else:
                # If no solution found, mark as potential singularity
                det_values[i, j] = 0
                singularity_map[i, j] = True
    
    return X, Y, det_values, singularity_map

# Analyze the singularity around the fully extended position
print("SINGULARITY ANALYSIS")
print("=" * 20)

# The arm is in singularity when it's fully extended (θ2 = 0)
# At this point, the end-effector position is (l1 + l2, 0)
singularity_pos = (l1 + l2, 0)

print(f"Known singularity configuration: fully extended")
print(f"  Joint angles: θ1=0, θ2=0")
print(f"  End-effector: ({l1+l2}, 0)")

# Calculate Jacobian at singularity configuration
J_at_singularity = jacobian_2dof_planar(0, 0, l1, l2)
det_at_singularity = np.linalg.det(J_at_singularity)
is_singular = detect_singularity(J_at_singularity)

print(f"  Jacobian: \n{J_at_singularity}")
print(f"  Determinant: {det_at_singularity:.6f}")
print(f"  Is singular: {is_singular}")
print()

# Analyze region around singularity
X, Y, det_values, sing_map = analyze_singularity_region(l1, l2, center=singularity_pos, radius=0.3)

# Plot the results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Singularity map
contour1 = ax1.contourf(X, Y, sing_map.astype(int), levels=1, colors=['red', 'white'], alpha=0.5)
ax1.set_title('Singularity Regions (Red = Singular)')
ax1.set_xlabel('X position')
ax1.set_ylabel('Y position')
ax1.grid(True, alpha=0.3)
ax1.axis('equal')

# Plot 2: Determinant values
contour2 = ax2.contourf(X, Y, det_values, levels=20, cmap='viridis')
ax2.set_title('Jacobian Determinant Values')
ax2.set_xlabel('X position')
ax2.set_ylabel('Y position')
ax2.grid(True, alpha=0.3)
ax2.axis('equal')
plt.colorbar(contour2, ax=ax2)

# Mark the exact singularity position
ax1.plot(singularity_pos[0], singularity_pos[1], 'k+', markersize=15, mew=3, label='Exact Singularity')
ax2.plot(singularity_pos[0], singularity_pos[1], 'k+', markersize=15, mew=3, label='Exact Singularity')
ax1.legend()
ax2.legend()

plt.tight_layout()
plt.show()

print(f"Analysis of region around fully extended configuration")
print(f"  Grid size: {res}x{res}")
print(f"  Region: ({center[0]-radius:.2f}, {center[1]-radius:.2f}) to ({center[0]+radius:.2f}, {center[1]+radius:.2f})")
print(f"  Points identified as singular: {np.sum(sing_map)} out of {res*res}")
print(f"  Min determinant in region: {np.min(det_values[np.nonzero(det_values)]):.8f}")
print(f"  Max determinant in region: {np.max(det_values):.6f}")

## 5. Damped Least Squares (DLS) Method

The Damped Least Squares method is a more robust approach that handles singularities better than the basic pseudoinverse method.

In [ ]:
def inverse_kinematics_dls_method(target_pos, initial_joints, link_lengths,
                               max_iterations=100, tolerance=1e-5, alpha=0.1, damping=0.01):
    """
    Solve inverse kinematics using the Damped Least Squares (DLS) method.
    This method adds damping to the singular values to improve performance near singularities.
    
    Args:
        target_pos: Desired end-effector position (x, y)
        initial_joints: Initial joint angles [theta1, theta2]
        link_lengths: Link lengths [l1, l2]
        max_iterations: Maximum number of iterations
        tolerance: Convergence tolerance
        alpha: Learning rate/scale factor
        damping: Damping factor (λ) for DLS
    
    Returns:
        Joint angles that achieve the target position, or None if failed
    """
    l1, l2 = link_lengths
    target_x, target_y = target_pos
    joints = np.array(initial_joints, dtype=float)
    
    for i in range(max_iterations):
        # Calculate current end-effector position
        curr_x, curr_y, _, _ = forward_kinematics_2dof(joints[0], joints[1], l1, l2)
        current_pos = np.array([curr_x, curr_y])
        
        # Calculate error
        error = np.array([target_x, target_y]) - current_pos
        
        # Check for convergence
        if np.linalg.norm(error) < tolerance:
            print(f"DLS method converged after {i+1} iterations")
            return joints
        
        # Calculate Jacobian
        J = jacobian_2dof_planar(joints[0], joints[1], l1, l2)
        
        # Calculate DLS update: dq = J^T * (J * J^T + λ^2 * I)^(-1) * dx
        # Or equivalently using SVD: dq = V * S^T * (S^2 + λ^2)^(-1) * U^T * dx
        U, s, Vt = np.linalg.svd(J, full_matrices=False)  # J = U * diag(s) * Vt
        
        # Apply damping: divide by (s^2 + λ^2)
        damped_s = s / (s**2 + damping**2)
        
        # Calculate joint update: dq = Vt.T * damped_s_diag * U.T * dx
        damped_s_diag = np.diag(damped_s)
        joint_change = Vt.T @ damped_s_diag @ U.T @ error * alpha
        
        # Update joint angles
        joints += joint_change
        
        # Limit excessive changes
        max_change = 0.2
        if np.linalg.norm(joint_change) > max_change:
            joints -= joint_change
            joints += joint_change * max_change / np.linalg.norm(joint_change)
    
    print(f"DLS method failed after {max_iterations} iterations. Final error: {np.linalg.norm(error):.6f}")
    return None

# Test DLS method
print("DAMPED LEAST SQUARES (DLS) METHOD")
print("=" * 35)

target = (1.5, 0.1)  # Near the singular region
initial_guess = [0.1, 0.1]

print(f"Target position: ({target[0]:.3f}, {target[1]:.3f})")
print(f"Close to singularity at ({l1+l2}, 0), distance: {abs(target[0] - (l1+l2)):.3f}")
print()

solution_dls = inverse_kinematics_dls_method(target, initial_guess, [l1, l2], damping=0.05)

if solution_dls is not None:
    print(f"DLS Solution: θ1={solution_dls[0]:.3f}, θ2={solution_dls[1]:.3f}")
    
    # Verify solution
    fk_x, fk_y, _, _ = forward_kinematics_2dof(solution_dls[0], solution_dls[1], l1, l2)
    final_error = np.sqrt((fk_x - target[0])**2 + (fk_y - target[1])**2)
    print(f"Verification: FK gives ({fk_x:.3f}, {fk_y:.3f}), Error: {final_error:.6f}")
    
    # Visualize
    plot_2dof_config(solution_dls[0], solution_dls[1], l1, l2, target, 
                    f'DLS Solution (damping=0.05) for target ({target[0]:.1f}, {target[1]:.1f})')
else:
    print("DLS solution not found")

## 6. Comparison of IK Methods

Let's compare the different IK methods we've implemented on a range of target positions.

In [ ]:
def compare_ik_methods(targets, initial_guess, link_lengths, methods):
    """
    Compare different IK methods on a set of target positions.
    
    Args:
        targets: List of target positions [(x1, y1), (x2, y2), ...]
        initial_guess: Initial joint angles
        link_lengths: Link lengths
        methods: List of method functions
    
    Returns:
        Dictionary with results for each method
    """
    results = {}
    
    for method_name, method_func in methods.items():
        print(f"Testing {method_name}...")
        method_results = []
        
        for target in targets:
            solution = method_func(target, initial_guess, link_lengths)
            
            if solution is not None:
                # Calculate error
                fk_x, fk_y, _, _ = forward_kinematics_2dof(solution[0], solution[1], *link_lengths)
                error = np.sqrt((fk_x - target[0])**2 + (fk_y - target[1])**2)
                method_results.append((solution, error, True))
            else:
                method_results.append((None, float('inf'), False))
        
        results[method_name] = method_results
        print(f"  Completed {method_name}")
        print()
    
    return results

# Define the target positions to test
test_targets = [
    (1.2, 0.5),   # General position
    (0.8, 0.8),   # More upward position
    (1.1, 0.1),   # Near horizontal (close to singularity)
    (0.5, 1.0),   # More upward and to the left
    (1.6, 0.2),   # Near the outer reach (close to singularity)
]

# Define the methods to compare
methods = {
    'Jacobian Transpose': lambda target, init, lengths: 
        inverse_kinematics_jacobian_method(target, init, lengths, max_iterations=100),
    'Jacobian Pseudoinverse': lambda target, init, lengths: 
        inverse_kinematics_pseudoinverse_method(target, init, lengths, max_iterations=100),
    'DLS (Damped Least Squares)': lambda target, init, lengths: 
        inverse_kinematics_dls_method(target, init, lengths, max_iterations=100, damping=0.02)
}

# Run comparison
print("COMPARISON OF IK METHODS")
print("=" * 25)

comparison_results = compare_ik_methods(test_targets, [0.1, 0.1], [l1, l2], methods)

# Print results table
print("RESULTS SUMMARY")
print("" + "Target Position" + " "*18 + "Method" + " "*17 + "Success" + " "*8 + "Final Error")
print("-" * 80)

for i, target in enumerate(test_targets):
    target_str = f"({target[0]:.2f}, {target[1]:.2f})"
    
    for method_name in methods.keys():
        solution, error, success = comparison_results[method_name][i]
        success_str = "YES" if success else "NO "
        print(f"{target_str:<20} {method_name:<20} {success_str:<8} {error:<10.6f}")
    
    print("  " + "-" * 76)

# Calculate overall statistics
print("\nOVERALL STATISTICS")
print("-" * 20)
for method_name in methods.keys():
    successful_solutions = [r for r in comparison_results[method_name] if r[2]]
    success_rate = len(successful_solutions) / len(test_targets) * 100
    
    if successful_solutions:
        avg_error = np.mean([r[1] for r in successful_solutions])
        max_error = np.max([r[1] for r in successful_solutions])
    else:
        avg_error = float('inf')
        max_error = float('inf')
    
    print(f"{method_name}:")
    print(f"  Success rate: {success_rate:.1f}% ({len(successful_solutions)}/{len(test_targets)})")
    print(f"  Average error (successful only): {avg_error:.6f}")
    print(f"  Max error (successful only): {max_error:.6f}")
    print()

## 7. Advanced Topic: Redundant Manipulators

For manipulators with more degrees of freedom than required task variables (redundant manipulators), there are infinitely many solutions to the IK problem. We can select among them using optimization criteria.

In [ ]:
def jacobian_3dof_planar(theta1, theta2, theta3, l1, l2, l3):
    """
    Calculate the Jacobian matrix for a 3-DOF planar manipulator.
    Even though it's 3-DOF, we're only controlling x, y (2 tasks).
    This makes it redundant and allows secondary objectives.
    
    Args:
        theta1, theta2, theta3: Joint angles
        l1, l2, l3: Link lengths
    
    Returns:
        2x3 Jacobian matrix
    """
    # Calculate intermediate angles
    s1 = np.sin(theta1)
    c1 = np.cos(theta1)
    s12 = np.sin(theta1 + theta2)
    c12 = np.cos(theta1 + theta2)
    s123 = np.sin(theta1 + theta2 + theta3)
    c123 = np.cos(theta1 + theta2 + theta3)
    
    # Construct the Jacobian
    J = np.zeros((2, 3))
    
    # dx/dtheta1
    J[0, 0] = -l1*s1 - l2*s12 - l3*s123
    # dx/dtheta2  
    J[0, 1] = -l2*s12 - l3*s123
    # dx/dtheta3
    J[0, 2] = -l3*s123
    # dy/dtheta1
    J[1, 0] = l1*c1 + l2*c12 + l3*c123
    # dy/dtheta2
    J[1, 1] = l2*c12 + l3*c123
    # dy/dtheta3
    J[1, 2] = l3*c123
    
    return J

def inverse_kinematics_redundant(target_pos, initial_joints, link_lengths,
                              max_iterations=100, tolerance=1e-5, alpha=0.1):
    """
    Inverse kinematics for redundant manipulator using null-space optimization.
    This method can incorporate secondary objectives like avoiding joint limits.
    
    Args:
        target_pos: Desired end-effector position (x, y)
        initial_joints: Initial joint angles [theta1, theta2, theta3]
        link_lengths: Link lengths [l1, l2, l3]
        max_iterations: Maximum number of iterations
        tolerance: Convergence tolerance
        alpha: Learning rate
    
    Returns:
        Joint angles that achieve the target position, or None if failed
    """
    l1, l2, l3 = link_lengths
    target_x, target_y = target_pos
    joints = np.array(initial_joints, dtype=float)
    
    # Joint limits for secondary objective (avoiding limits)
    joint_limits = [(-np.pi, np.pi), (-np.pi, np.pi), (-np.pi, np.pi)]
    
    for i in range(max_iterations):
        # Calculate current end-effector position
        # For the 3-DOF case, we'll calculate forward kinematics as:
        x = l1*np.cos(joints[0]) + l2*np.cos(joints[0] + joints[1]) + l3*np.cos(joints[0] + joints[1] + joints[2])
        y = l1*np.sin(joints[0]) + l2*np.sin(joints[0] + joints[1]) + l3*np.sin(joints[0] + joints[1] + joints[2])
        current_pos = np.array([x, y])
        
        # Calculate error
        error = np.array([target_x, target_y]) - current_pos
        
        # Check for convergence
        if np.linalg.norm(error) < tolerance:
            print(f"Redundant IK converged after {i+1} iterations")
            return joints
        
        # Calculate Jacobian
        J = jacobian_3dof_planar(joints[0], joints[1], joints[2], l1, l2, l3)
        
        # Calculate pseudoinverse
        J_pinv = np.linalg.pinv(J)
        
        # Primary task: move toward target
        primary_change = J_pinv @ error * alpha
        
        # Secondary task: move away from joint limits
        # Calculate distance to nearest limit for each joint
        joint_centering = np.zeros(3)
        for j_idx in range(3):
            lower, upper = joint_limits[j_idx]
            center = (lower + upper) / 2
            current = joints[j_idx]
            # If we're moving toward a limit, apply corrective motion
            if current > center:
                # Closer to upper limit, move toward center
                dist_to_limit = upper - current
                if dist_to_limit < 0.5:  # If within 0.5 rad of limit
                    joint_centering[j_idx] = -0.1 * (0.5 - dist_to_limit)
            else:
                # Closer to lower limit, move toward center
                dist_to_limit = current - lower
                if dist_to_limit < 0.5:  # If within 0.5 rad of limit
                    joint_centering[j_idx] = 0.1 * (0.5 - dist_to_limit)
        
        # Calculate null-space projector: I - J_pinv @ J
        I = np.eye(3)
        null_projector = I - J_pinv @ J
        
        # Apply secondary objective in null space
        secondary_change = null_projector @ joint_centering
        
        # Combine primary and secondary objectives
        total_change = primary_change + 0.1 * secondary_change
        
        # Update joint angles
        joints += total_change
        
        # Limit excessive changes
        max_change = 0.2
        if np.linalg.norm(total_change) > max_change:
            joints -= total_change
            joints += total_change * max_change / np.linalg.norm(total_change)
    
    print(f"Redundant IK failed after {max_iterations} iterations. Final error: {np.linalg.norm(error):.6f}")
    return None

# Test redundant manipulator
print("REDUNDANT MANIPULATOR INVERSE KINEMATICS")
print("=" * 45)

# Create a 3-DOF manipulator
l1, l2, l3 = 0.6, 0.5, 0.4
print(f"3-DOF Manipulator: L1={l1}, L2={l2}, L3={l3}".format(l1=l1, l2=l2, l3=l3))
print(f"Max reach: {l1+l2+l3}, Min reach: {abs(l1-l2-l3)}")
print()

target = (1.0, 0.8)
initial = [0.1, 0.1, 0.1]

print(f"Target position: ({target[0]:.3f}, {target[1]:.3f})")
print(f"Initial joints: {initial}")

redundant_solution = inverse_kinematics_redundant(target, initial, [l1, l2, l3])

if redundant_solution is not None:
    print(f"\nSolution found:")
    print(f"  θ1={redundant_solution[0]:.3f}, θ2={redundant_solution[1]:.3f}, θ3={redundant_solution[2]:.3f}")
    
    # Verify
    x = l1*np.cos(redundant_solution[0]) + l2*np.cos(redundant_solution[0] + redundant_solution[1]) + \
        l3*np.cos(redundant_solution[0] + redundant_solution[1] + redundant_solution[2])
    y = l1*np.sin(redundant_solution[0]) + l2*np.sin(redundant_solution[0] + redundant_solution[1]) + \
        l3*np.sin(redundant_solution[0] + redundant_solution[1] + redundant_solution[2])
    
    error = np.sqrt((x - target[0])**2 + (y - target[1])**2)
    print(f"Verification: FK gives ({x:.3f}, {y:.3f}), Error: {error:.6f}")
    
    # Check joint limit violations
    limit_violations = []
    for j_idx, (angle, (low, high)) in enumerate(zip(redundant_solution, [(-np.pi, np.pi)]*3)):
        if angle < low or angle > high:
            limit_violations.append(f"J{j_idx+1}={angle:.3f} (lim=[{low:.3f}, {high:.3f}])")
    
    if limit_violations:
        print(f"Joint limit violations: {', '.join(limit_violations)}")
    else:
        print(f"All joints within limits")
else:
    print("\nNo solution found")

## 8. Summary and Key Insights

Throughout this notebook, we've explored various inverse kinematics methods:

1. **Analytical Methods**: Exact solutions for simple configurations; limited in applicability
2. **Numerical Methods**: General solution approach using Jacobians
   - Jacobian transpose: Simple but struggles near singularities
   - Jacobian pseudoinverse: Better behavior but still problematic at singularities
   - Damped Least Squares: Robust to singularities through regularization
3. **Redundant Manipulators**: Special case with extra DOF allowing for optimization criteria

Choosing the right method depends on your specific application, robot complexity, and computational requirements.

In [ ]:
# Final summary
print("KEY INSIGHTS FROM INVERSE KINEMATICS")
print("=" * 40)
print("1. Inverse Kinematics finds joint angles for desired end-effector position")
print("2. Analytical solutions exist for simple robots but not for complex ones")
print("3. Numerical methods are general-purpose but require good initialization")
print("4. Singularity handling is crucial for reliable robot control")
print("5. Redundant robots have infinite solutions; secondary objectives help select")
print("6. Different methods have trade-offs in accuracy, stability, and computational cost")